In [19]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from cleantext import clean

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.metrics import classification_report, f1_score, accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import r2_score
import random

In [2]:
data = pd.read_csv("EMB_all.csv")
data = data.drop('Unnamed: 0',1)
data

,userName,date,content,score,if_apple,if_useful,crush
0,Jocelyn Sy,2022-10-27 18:10:20,fun,5,0,0.0,0.0
1,crystal may,2022-10-27 12:17:07,aw matter game close progress spent coin gem l...,1,0,1.0,0.0
2,Coreter,2022-10-27 07:07:17,cool,5,0,0.0,0.0
3,Jordan Christie,2022-10-26 15:11:40,use love game recent new go sync progress acco...,2,0,1.0,0.0
4,Heather Nicole,2022-10-26 03:42:00,love game dont know shut time clear make sure ...,4,0,1.0,0.0
...,...,...,...,...,...,...,...
170696,ty lol ty,2022-09-26 07:38:26,fun,5,1,0.0,0.0
170697,Lunas GiGi,2022-09-05 14:14:42,NaN,5,1,0.0,0.0
170698,pogdw,2022-09-20 21:43:42,fun,4,1,0.0,0.0
170699,arajuju,2022-08-03 12:04:10,love,5,1,0.0,0.0


In [3]:
label_data = pd.read_csv("game_reviews/labelled_all.csv")
label_data = label_data.drop('Unnamed: 0',1)
label_data

,userName,date,content,score,if_apple,if_useful,mechanism,ad,money,UI/UX,event,keyboard,IP,time/life,customer service,crush,data,system upgrad,connection,other-tech
0,Cre8tiv99,2019-03-11 17:34:05,"Hey guys, this is Bryce “Cre8tiv” Demby. Love ...",3,1,1.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,eclaitse25,2022-04-05 22:28:14,I don't want to bore you with a long review lo...,4,1,1.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Jackieee003,2020-11-19 03:12:21,Fav game!! Love the challenges and Disney sett...,5,1,1.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tab the great,2021-11-13 03:55:52,This game is great for all ages and fun to play!,5,1,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Roberts232,2018-03-08 06:26:19,Special Disney emoji earned through play.,3,1,1.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,A Google user,2018-04-12 06:40:42,I love this game,5,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,A Google user,2019-01-04 13:55:12,I have been playing Emoji Blitz for awhile now...,3,0,1.0,0.0,0.0,-1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,A Google user,2017-09-20 22:58:51,Love this game!!!,5,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,Chandresh Panchal,2020-10-30 02:53:09,How to chat with Disney emojis,5,0,1.0,0.0,0.0,0.0,0.0,0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REMOVE_NUM = re.compile('[\d+]')
STOPWORDS = set(stopwords.words('english'))
en_words = set(nltk.corpus.words.words())

def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    # lowercase text
    text = text.lower() 

    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    
    # Remove the XXXX values
    text = text.replace('x', '') 
    
    # Remove white space
    text = REMOVE_NUM.sub('', text)

    #  delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('', text) 

    # delete stopwords from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 

    # Stemming the words
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    # removing non-English words
    text = ' '.join(word for word in text.split() if word in en_words)
    
    return text

In [5]:
data['content'] = data['content'].astype(str)
label_data['conent'] = label_data['content'].astype(str)
data['content'] = data['content'].apply(clean_text)

In [6]:
label_data['content'] = label_data['content'].apply(clean_text)

In [7]:
data

,userName,date,content,score,if_apple,if_useful,crush
0,Jocelyn Sy,2022-10-27 18:10:20,fun,5,0,0.0,0.0
1,crystal may,2022-10-27 12:17:07,aw matter game close progress spent coin gem l...,1,0,1.0,0.0
2,Coreter,2022-10-27 07:07:17,cool,5,0,0.0,0.0
3,Jordan Christie,2022-10-26 15:11:40,use love game recent new go sync progress acco...,2,0,1.0,0.0
4,Heather Nicole,2022-10-26 03:42:00,love game dont know shut time clear make sure ...,4,0,1.0,0.0
...,...,...,...,...,...,...,...
170696,ty lol ty,2022-09-26 07:38:26,fun,5,1,0.0,0.0
170697,Lunas GiGi,2022-09-05 14:14:42,nan,5,1,0.0,0.0
170698,pogdw,2022-09-20 21:43:42,fun,4,1,0.0,0.0
170699,arajuju,2022-08-03 12:04:10,love,5,1,0.0,0.0


In [8]:
label_data

,userName,date,content,score,if_apple,if_useful,mechanism,ad,money,UI/UX,...,keyboard,IP,time/life,customer service,crush,data,system upgrad,connection,other-tech,conent
0,Cre8tiv99,2019-03-11 17:34:05,hey guy love game yet one minor got high score...,3,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Hey guys, this is Bryce “Cre8tiv” Demby. Love ..."
1,eclaitse25,2022-04-05 22:28:14,dont want bore long review hard get new,4,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I don't want to bore you with a long review lo...
2,Jackieee003,2020-11-19 03:12:21,game love set,5,1,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fav game!! Love the challenges and Disney sett...
3,tab the great,2021-11-13 03:55:52,game great age fun play,5,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,This game is great for all ages and fun to play!
4,Roberts232,2018-03-08 06:26:19,special earn play,3,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Special Disney emoji earned through play.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,A Google user,2018-04-12 06:40:42,love game,5,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I love this game
4996,A Google user,2019-01-04 13:55:12,play blitz complaint option use gem power want,3,0,1.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I have been playing Emoji Blitz for awhile now...
4997,A Google user,2017-09-20 22:58:51,love game,5,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Love this game!!!
4998,Chandresh Panchal,2020-10-30 02:53:09,chat,5,0,1.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,How to chat with Disney emojis


In [9]:
train = label_data.copy()
test = data.copy()

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(train['content'])
X_test_vectors_tfidf = tfidf_vectorizer.transform(test['content'])

df_X_train = pd.DataFrame(X_train_vectors_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
df_X_train[["if_useful"]] = train[["if_useful"]]
df_X_test = pd.DataFrame(X_test_vectors_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
df_X_test[["if_useful"]] = test[["if_useful"]]

# if_useful

In [10]:
clf = SGDClassifier()
clf = clf.fit(df_X_train, label_data['data'])

In [11]:
y_pred = clf.predict(df_X_test)

In [12]:
data['data'] = y_pred
data

,userName,date,content,score,if_apple,if_useful,crush,data
0,Jocelyn Sy,2022-10-27 18:10:20,fun,5,0,0.0,0.0,0.0
1,crystal may,2022-10-27 12:17:07,aw matter game close progress spent coin gem l...,1,0,1.0,0.0,-1.0
2,Coreter,2022-10-27 07:07:17,cool,5,0,0.0,0.0,0.0
3,Jordan Christie,2022-10-26 15:11:40,use love game recent new go sync progress acco...,2,0,1.0,0.0,0.0
4,Heather Nicole,2022-10-26 03:42:00,love game dont know shut time clear make sure ...,4,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
170696,ty lol ty,2022-09-26 07:38:26,fun,5,1,0.0,0.0,0.0
170697,Lunas GiGi,2022-09-05 14:14:42,nan,5,1,0.0,0.0,0.0
170698,pogdw,2022-09-20 21:43:42,fun,4,1,0.0,0.0,0.0
170699,arajuju,2022-08-03 12:04:10,love,5,1,0.0,0.0,0.0


In [13]:
data.to_csv("EMB_all.csv")